In [2]:
import pandas as pd

In [3]:
data= pd.read_csv('../../Data/rawData/PNW/PNW_angiosperm_photos.csv', sep='\t')
processed_data = data[["photo_url_square", "extension", "photo_id", "latitude", "longitude", "observed_on", "taxon_name", "rank", "ancestry"]].dropna()

In [4]:
processed_data

,photo_url_square,extension,photo_id,latitude,longitude,observed_on,taxon_name,rank,ancestry
0,https://inaturalist-open-data.s3.amazonaws.com...,jpg,20187,46.789153,-121.735554,2010-08-13,Polemonium californicum,species,48460/47126/211194/47125/47124/47181/48932/48933
1,https://inaturalist-open-data.s3.amazonaws.com...,jpg,23406,45.918975,-123.974276,2009-09-15,Baccharis pilularis consanguinea,subspecies,48460/47126/211194/47125/47124/47605/47604/632...
2,https://inaturalist-open-data.s3.amazonaws.com...,jpg,24243,43.833109,-124.152095,2011-03-05,Garrya elliptica,species,48460/47126/211194/47125/47124/53400/53401/53402
3,https://inaturalist-open-data.s3.amazonaws.com...,jpg,24244,43.822140,-124.156334,2011-03-05,Cytisus scoparius,species,48460/47126/211194/47125/47124/47123/47122/507...
4,https://inaturalist-open-data.s3.amazonaws.com...,jpg,24245,43.818100,-124.161491,2011-03-05,Claytonia exigua glauca,subspecies,48460/47126/211194/47125/47124/47366/71417/529...
...,...,...,...,...,...,...,...,...,...
2819363,https://inaturalist-open-data.s3.amazonaws.com...,jpeg,348207502,46.784711,-121.748353,2023-09-19,Symphyotrichum foliaceum,species,48460/47126/211194/47125/47124/47605/47604/632...
2819364,https://inaturalist-open-data.s3.amazonaws.com...,jpeg,348207533,46.784711,-121.748353,2023-09-19,Symphyotrichum foliaceum,species,48460/47126/211194/47125/47124/47605/47604/632...
2819365,https://inaturalist-open-data.s3.amazonaws.com...,jpeg,348207540,46.784711,-121.748353,2023-09-19,Symphyotrichum foliaceum,species,48460/47126/211194/47125/47124/47605/47604/632...
2819366,https://inaturalist-open-data.s3.amazonaws.com...,jpeg,348207528,46.784711,-121.748353,2023-09-19,Symphyotrichum foliaceum,species,48460/47126/211194/47125/47124/47605/47604/632...


In [5]:
def get_year(date_str):
    return int(date_str[0:4])

def get_month(date_str):
    return int(date_str[5:7])

def get_day(date_str):
    return int(date_str[8:])

processed_data["year"] = processed_data["observed_on"].apply(get_year)
processed_data["month"] = processed_data["observed_on"].apply(get_month)
processed_data["day"] = processed_data["observed_on"].apply(get_day)
processed_data = processed_data.drop("observed_on", axis=1)

In [6]:
ob_count_list = []
year_list = processed_data["year"].unique()
year_list.sort()
for year in year_list:
    ob_count_list.append((year, processed_data[processed_data["year"] == year].shape[0]))

Let's focus on images after 2013. 

In [7]:
processed_data = processed_data[processed_data["year"] > 2013]

In [8]:
#We need small photos rather than square photos in order to fit our ResNet model. 
def get_small_url(url):
    url_list = url.split(".")
    last_word = url_list[-2]
    new_last = last_word[:-6] + "small"
    url_list[-2] = new_last
    new_url = ".".join(url_list)
    return new_url

processed_data["small_url"] = processed_data["photo_url_square"].apply(get_small_url)
processed_data["file_name"] = processed_data["photo_id"].astype(str) + "." + processed_data["extension"]

In [9]:
sample = processed_data.sample(2002000, random_state=26)

In [10]:
len(sample["photo_id"].unique())

2001734

In [14]:
sample.to_csv('../../Data/processedData/PNW/all_PNW.csv', index=False)

In [12]:
#Here is code to create a batch file to download all the necessary images. 
'''
for k in range(10):
    n = 1001
    m = 200
    with open("../../Data/processedData/PNW/Bats/allPhotos" + str(k) + ".bat", "w") as file:
        for j in range(n):
            file.write('curl')
            for i in range(m):
                idx = i + m*j + m*n*k 
                url = sample['small_url'].iloc[idx]
                file_name = sample['file_name'].iloc[idx]
                path = 'D:/PNW_Angiosperms/all_photos_' + str(k) + '/'
                full_name = path + file_name
                file.write(' -o ' + full_name + ' ' + url)
            file.write('\n')
'''

'\nfor k in range(10):\n    n = 1001\n    m = 200\n    with open("../../Data/processedData/PNW/Bats/allPhotos" + str(k) + ".bat", "w") as file:\n        for j in range(n):\n            file.write(\'curl\')\n            for i in range(m):\n                idx = i + m*j + m*n*k \n                url = sample[\'small_url\'].iloc[idx]\n                file_name = sample[\'file_name\'].iloc[idx]\n                path = \'D:/PNW_Angiosperms/all_photos_\' + str(k) + \'/\'\n                full_name = path + file_name\n                file.write(\' -o \' + full_name + \' \' + url)\n            file.write(\'\n\')\n'